<http://www.atmarkit.co.jp/ait/articles/1310/02/news006.html>
# 学習塾を運営するのに最適なのはどこ？ オープンデータを活用して実践的なスキルを身に付ける (1/3)
No5とNo6の間の特別編
政府や行政が主導して国内でも環境が整いつつあるオープンデータの活用。今回は特別編として、オープンデータを活用した実践的な分析を展開します。

データの統合や変換、クレンジングを行なう練習

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')


### 千葉市の町丁別年齢別人口を活用
<http://www.city.chiba.jp/sogoseisaku/sogoseisaku/tokei/jinkou-jyuki.html#chobetsu>

※記事とデータファイルの形式が変わっている
記事では区別、年度別にExcelファイルがあるが、現時点では区別のファイルがあって年度はExcelシートがわかれている

（あらかじめダウンロードしたファイルをdataディレクトリに配置済み）

In [2]:
# 例として千葉市中央区の年度別年齢別人口データ
xls = pd.ExcelFile("data/tyu_tyo03.xls")

# Excelデータを開いてみれば分かるが、クロス集計済みのExcelファイルなのでプログラムで分析するには使いづらい
# クロス集計前のフラットなデータに戻すためにクレンジングを行なう
#  フラットなデータとは？このような感じ
# →フラットなデータ構造にすることで指定した条件の人口を簡単に選択できる

pd.DataFrame(columns=['year', 'month', 'ward', 'town', 'gender', 'age', 'num'], 
            data=[
        [2015, 3, 'A区', 'X町', '男', 1, 10],
        [2015, 3, 'A区', 'X町', '女', 1, 11],
        [2015, 3, 'A区', 'X町', '男', 2, 12],
        [2015, 3, 'A区', 'X町', '男', 2, 13],
                 ])

,year,month,ward,town,gender,age,num
0,2015,3,A区,X町,男,1,10
1,2015,3,A区,X町,女,1,11
2,2015,3,A区,X町,男,2,12
3,2015,3,A区,X町,男,2,13


In [3]:
#先頭シート(平成27年)の2行読み飛ばして（3行目ヘッダ、4行目以降データ）B:DE列をパース
df = xls.parse(xls.sheet_names[0], skiprows=2, parse_cols="B:DE")
df.head()

,Unnamed: 0,町丁名,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107
0,中央区,総数,総数,201721,25514,131577,44630,1742,1702,1690,...,343,272,231,182,145,100,85,54,35,83
1,NaN,NaN,男,101865,13127,68816,19922,903,904,873,...,83,59,51,48,26,21,19,7,2,16
2,NaN,NaN,女,99856,12387,62761,24708,839,798,817,...,260,213,180,134,119,79,66,47,33,67
3,中央区,青葉町,総数,2919,445,1930,544,23,21,28,...,5,0,2,3,2,1,0,0,0,1
4,NaN,NaN,男,1389,200,955,234,9,11,12,...,1,0,0,1,0,0,0,0,0,0


In [4]:
#区のデータは列=0（ilocで列指定)
df.iloc[:, 0].head(5) 

0    中央区
1    NaN
2    NaN
3    中央区
4    NaN
Name: Unnamed: 0, dtype: object

In [5]:
# データの空白を埋める
df.iloc[:, 0].fillna(method='ffill').head(5)  #ffillは前のデータで空白を埋める

0    中央区
1    中央区
2    中央区
3    中央区
4    中央区
Name: Unnamed: 0, dtype: object

In [6]:
# 余分なスペースを除去（まれにセルに空白が含まれているものがある）
# applyで全データに関数の実行結果を適用する
df.iloc[:, 0].fillna(method='ffill').apply(lambda x:x.strip()).head(5)

0    中央区
1    中央区
2    中央区
3    中央区
4    中央区
Name: Unnamed: 0, dtype: object

In [7]:
# 年度はシート名から決定
import re
xls.sheet_names[0] # →平成27年
int(re.sub(r'平成(\d+)年', r'\1', xls.sheet_names[0]))+1988   # 27+1988 = 2015

2015

In [8]:
# 以上を踏まえて加工した列を、作成した空のデータフレームに追加
df['year'] = int(re.sub(r'平成(\d+)年', r'\1', xls.sheet_names[0]))+1988
df['month'] = 3
df['ward'] = df.iloc[:,0].fillna(method='ffill').apply(lambda x:x.strip())
df['town'] = df.iloc[:,1].fillna(method='ffill').apply(lambda x:x.strip())
df['gender'] = df.iloc[:,2].fillna(method='ffill').apply(lambda x:x.strip())

In [9]:
df.head()

,Unnamed: 0,町丁名,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,year,month,ward,town,gender
0,中央区,総数,総数,201721,25514,131577,44630,1742,1702,1690,...,100,85,54,35,83,2015,3,中央区,総数,総数
1,NaN,NaN,男,101865,13127,68816,19922,903,904,873,...,21,19,7,2,16,2015,3,中央区,総数,男
2,NaN,NaN,女,99856,12387,62761,24708,839,798,817,...,79,66,47,33,67,2015,3,中央区,総数,女
3,中央区,青葉町,総数,2919,445,1930,544,23,21,28,...,1,0,0,0,1,2015,3,中央区,青葉町,総数
4,NaN,NaN,男,1389,200,955,234,9,11,12,...,0,0,0,0,0,2015,3,中央区,青葉町,男


In [10]:
# 町名=総数の行は削除
# さらにyearよりも前の列は不要なので、使用するのはindex=3以降の行で、year以降の列のみを選択した結果

df2=df.ix[3:, 'year':]
df2.head()

,year,month,ward,town,gender
3,2015,3,中央区,青葉町,総数
4,2015,3,中央区,青葉町,男
5,2015,3,中央区,青葉町,女
6,2015,3,中央区,赤井町,総数
7,2015,3,中央区,赤井町,男


In [11]:
# gender=総数の行は集計行なので除去
df2=df2[df2['gender'] != '総数']
df2.head()

,year,month,ward,town,gender
4,2015,3,中央区,青葉町,男
5,2015,3,中央区,青葉町,女
7,2015,3,中央区,赤井町,男
8,2015,3,中央区,赤井町,女
10,2015,3,中央区,旭町,男


In [12]:

# df['age'], df['num']に年齢別の人口を設定し、必要な行・列だけのdf_wardを作成する
# df['num']は、dfの各年齢の列データを設定する
# とりあえず年齢0のデータでやってみる
age=0
df['age'] = age
df['num'] = df.iloc[:,age+7].fillna(0).apply(lambda x: int(x))   #年齢列を数値化してdf['num']にする
df2=df.ix[3:, 'year':] #列の絞り込み
df2=df2[df2['gender'] != '総数']  #総数行は不要
# クロス集計前のフラットなデータdf_wardに戻すために以下のような構造を作成する
df_ward = pd.DataFrame(columns=['year', 'month', 'ward', 'town', 'gender', 'age', 'num'], data=df2)
    

In [13]:
# 最終的にできあがったdf_ward(中央区平成27年度の町村別 性別別の0歳人口)を確認
df_ward.head(10)

,year,month,ward,town,gender,age,num
4,2015,3,中央区,青葉町,男,0,9
5,2015,3,中央区,青葉町,女,0,14
7,2015,3,中央区,赤井町,男,0,10
8,2015,3,中央区,赤井町,女,0,6
10,2015,3,中央区,旭町,男,0,5
11,2015,3,中央区,旭町,女,0,4
13,2015,3,中央区,市場町,男,0,3
14,2015,3,中央区,市場町,女,0,1
16,2015,3,中央区,亥鼻１丁目,男,0,1
17,2015,3,中央区,亥鼻１丁目,女,0,0


In [14]:
#ここまでの処理は、中央区ファイルの平成27年のシートだけの0歳だけを対象に行った。
# 以上のプロセスを踏まえて、て全ての区（ファイル）、すべての年度（シート）、全年齢についてデータを整形する
# また整形したデータをCSVファイルとして保存する、df_wardに追加していくとだんだん遅くなるので、CSVファイルとして追記して作成する。

import os
import re
import logging
import datetime

# 出力用ファイルが存在したら削除（繰り返し実行した場合のため）
if os.path.exists("data/all_tyo.csv"):
    os.unlink("data/all_tyo.csv")

logging.root.setLevel(logging.DEBUG)  #時間がかかるのでログ出力で状況確認する

wardlist = ['han', 'ina', 'mid', 'mih', 'tyu', 'wak'] #区のリスト

for ward in wardlist: #全ての区
    filename = "data/%s_tyo03.xls" % ward
    xls = pd.ExcelFile(filename)
    for sheet_name in xls.sheet_names: #全ての年度
        df = xls.parse(sheet_name, skiprows=2, parse_cols="B:DE")
        year = int(re.sub(r'平成(\d+)年', r'\1', sheet_name))+1988  #正規表現で年号取り出し西暦変換
        df['year'] = year
        df['month'] = 3
        df['ward'] = df.iloc[:,0].fillna(method='ffill').apply(lambda x:x.strip())
        df['town'] = df.iloc[:,1].fillna(method='ffill').apply(lambda x:x.strip())
        df['gender'] = df.iloc[:,2].fillna(method='ffill').apply(lambda x:x.strip())
        
        for age in range(0, 101): # 0~100歳
            df['age'] = age
            df['num'] = df.iloc[:,age+7].fillna(0).apply(lambda x: int(x)) 
            df_ward = pd.DataFrame(columns=['year', 'month', 'ward', 'town', 'gender', 'age', 'num'], data=df.ix[3:, 'year':])
            df_ward = df_ward[df_ward.gender != '総数']  #gender=総数の行を削除
            df_ward.to_csv('data/all_tyo.csv' , index=False, encoding='utf8', mode="a", header=False)   #追記でCSVに書き込み
            
        logging.debug("%s\t%s-%d\n", datetime.datetime.now(), filename, year)
        
    xls.close()
logging.debug("finish")
xls.close()

DEBUG:root:2016-02-11 01:44:55.309303	data/han_tyo03.xls-2015

DEBUG:root:2016-02-11 01:44:56.041960	data/han_tyo03.xls-2014

DEBUG:root:2016-02-11 01:44:56.708082	data/han_tyo03.xls-2013

DEBUG:root:2016-02-11 01:44:57.387364	data/han_tyo03.xls-2012

DEBUG:root:2016-02-11 01:44:58.038602	data/han_tyo03.xls-2011

DEBUG:root:2016-02-11 01:44:58.717686	data/han_tyo03.xls-2010

DEBUG:root:2016-02-11 01:44:59.393479	data/han_tyo03.xls-2009

DEBUG:root:2016-02-11 01:45:00.073680	data/han_tyo03.xls-2008

DEBUG:root:2016-02-11 01:45:00.772448	data/han_tyo03.xls-2007

DEBUG:root:2016-02-11 01:45:01.500652	data/han_tyo03.xls-2006

DEBUG:root:2016-02-11 01:45:02.260244	data/han_tyo03.xls-2005

DEBUG:root:2016-02-11 01:45:02.974489	data/han_tyo03.xls-2004

DEBUG:root:2016-02-11 01:45:03.640460	data/han_tyo03.xls-2003

DEBUG:root:2016-02-11 01:45:04.421458	data/han_tyo03.xls-2002

DEBUG:root:2016-02-11 01:45:05.124937	data/han_tyo03.xls-2001

DEBUG:root:2016-02-11 01:45:05.779129	data/han_tyo03.xl

In [15]:
# csvから読み直して確認(2,156,552件)
dfall = pd.read_csv('data/all_tyo.csv', names=['year', 'month', 'ward', 'town', 'gender', 'age', 'num'], encoding='UTF-8')
dfall.count()

year      2156552
month     2156552
ward      2156552
town      2156552
gender    2156552
age       2156552
num       2156552
dtype: int64